In [2]:
%load_ext autoreload
%autoreload 2

import torch
from mvn.utils import multiview

from mvn.datasets.cmu import CMUSceneDataset
from mvn.datasets.human36m import Human36MMultiViewDataset, Human36MSingleViewDataset
from torch.utils.data import DataLoader

import os
from itertools import islice
from tqdm import tqdm_notebook
import pickle


import numpy as np
import cv2
import time

from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from mvn.utils.img import image_batch_to_numpy, denormalize_image
from mvn.models.temporal import TemporalModel, TemporalModelBase
from mvn.models.basic import MVNetVolumetricHybridResNet_2, MVNetVolumetricHybridResNet_temporal
from mvn.utils.multiview import project_3d_points_to_image_plane_without_distortion
from mvn.utils.vis import draw_2d_pose

from mvn.utils import img
from mvn.utils import multiview
from mvn.utils import op
from mvn.utils import vis
from mvn.utils import misc
from mvn.utils import cfg

from mvn.datasets import cmu, kth
from mvn.datasets import utils as dataset_utils
from mvn.datasets.human36m import Human36MMultiViewDataset, Human36MSingleViewDataset

from train import setup_human36m_dataloaders

from IPython.core.debugger import set_trace

from matplotlib import pyplot as plt
from warnings import filterwarnings
filterwarnings("ignore")

%matplotlib inline

retval = {
    'subject_names': ['S1', 'S5', 'S6', 'S7', 'S8', 'S9', 'S11'],
    'camera_names': ['54138969', '55011271', '58860488', '60457274'],
    'action_names': [
        'Directions-1', 'Directions-2',
        'Discussion-1', 'Discussion-2',
        'Eating-1', 'Eating-2',
        'Greeting-1', 'Greeting-2',
        'Phoning-1', 'Phoning-2',
        'Posing-1', 'Posing-2',
        'Purchases-1', 'Purchases-2',
        'Sitting-1', 'Sitting-2',
        'SittingDown-1', 'SittingDown-2',
        'Smoking-1', 'Smoking-2',
        'TakingPhoto-1', 'TakingPhoto-2',
        'Waiting-1', 'Waiting-2',
        'Walking-1', 'Walking-2',
        'WalkingDog-1', 'WalkingDog-2',
        'WalkingTogether-1', 'WalkingTogether-2']
}
# import nonechucks